In [1]:
import os
from PIL import Image
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import tqdm

class CustomDatasetYOLO:
    def __init__(self, folder_path, image_size=416, transforms=None):
        self.folder_path = folder_path
        self.transforms = transforms
        self.image_size = image_size
        self.data_pairs = self._load_data_pairs()

    def _load_data_pairs(self):
        image_files = []
        json_files = {}
        data_pairs = []

        for root, _, files in os.walk(self.folder_path):
            for file in files:
                if file.endswith((".jpg", ".jpeg", ".png")):
                    image_files.append(os.path.join(root, file))
                elif file.endswith(".json"):
                    json_files[os.path.splitext(file)[0]] = os.path.join(root, file)

        for image_file_path in tqdm.tqdm(image_files, desc="Loading data pairs"):
            image_name = os.path.splitext(os.path.basename(image_file_path))[0]

            # 해당 이미지 파일과 매칭되는 JSON 파일 찾기
            if image_name in json_files:
                json_file_path = json_files[image_name]
                data_pairs.append((image_file_path, json_file_path))
            else:
                print(f"JSON file not found for image: {image_file_path}")

        return data_pairs

    def resize_image_and_annotations(self, image_path, annotations, categories, target_size):
        img = Image.open(image_path)
        img_resized = img.resize(target_size, Image.LANCZOS)  # 이미지 크기 조정

        width_ratio = target_size[0] / img.width  # 너비 비율 계산
        height_ratio = target_size[1] / img.height  # 높이 비율 계산

        # 한글에서 영어로 라벨을 매핑하는 딕셔너리 (예시)
        label_mapping = {'화방': 1, '줄기': 2, '잎': 3, '열매': 0}

        labels = []
        for ann in annotations:
            x, y, w, h = ann['bbox']
            category_id = ann['category_id']
            category_name = categories.get(category_id, '')

            # 라벨 매핑
            label = label_mapping.get(category_name, -1)
            if label == -1:
                # 처리할 수 없는 경우 스킵
                continue

            # 조정된 이미지의 바운딩 박스 좌표 계산
            x_resized = x * width_ratio
            y_resized = y * height_ratio
            w_resized = w * width_ratio
            h_resized = h * height_ratio

            # YOLO 형식의 정규화된 좌표 계산
            x_center = (x_resized + w_resized / 2) / target_size[0]
            y_center = (y_resized + h_resized / 2) / target_size[1]
            box_width = w_resized / target_size[0]
            box_height = h_resized / target_size[1]

            # YOLO 형식 문자열로 변환
            label_str = f"{label} {x_center} {y_center} {box_width} {box_height}"
            labels.append(label_str)

        return img_resized, labels

    def create_label_files(self):
        def process_image(image_path, json_path, split):
            parent_folder = os.path.join(self.folder_path, split)
            images_folder = os.path.join(parent_folder, 'images')
            labels_folder = os.path.join(parent_folder, 'labels')
            os.makedirs(images_folder, exist_ok=True)
            os.makedirs(labels_folder, exist_ok=True)

            img_name = os.path.splitext(os.path.basename(image_path))[0]
            txt_file = os.path.join(labels_folder, f"{img_name}.txt")
            img_resized_path = os.path.join(images_folder, f"{img_name}.jpg")

            with open(json_path, 'r') as json_file:
                json_data = json.load(json_file)

            # 라벨이 없는 경우 패스
            if not json_data.get('annotations'):
                print(f"No annotations found for {image_path}, skipping...")
                return

            # categories 정보를 가져옴
            categories = {cat['id']: cat['name'] for cat in json_data['categories']}

            img_resized, labels = self.resize_image_and_annotations(image_path, json_data['annotations'], categories, (self.image_size, self.image_size))

            img_resized.save(img_resized_path)

            with open(txt_file, 'w') as f:
                for label in labels:
                    f.write(f"{label}\n")

        with ThreadPoolExecutor() as executor:
            futures = []
            for image_path, json_path in self.data_pairs:
                split = image_path.split(os.sep)[-3]  # 'train', 'val', 'test'
                futures.append(executor.submit(process_image, image_path, json_path, split))

            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    print(f"Exception occurred: {e}")

        print("YOLO 라벨 파일 생성이 완료되었습니다.")


In [2]:
if __name__ == "__main__":
    dataset = CustomDatasetYOLO('D:/NOA/시설작물모델/output/datasets', image_size=640)
    dataset.create_label_files()

Loading data pairs: 100%|██████████| 15081/15081 [00:00<00:00, 209947.69it/s]
